The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>
Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review

#### <u>Objective </u>:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).


In [2]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
# LOADING the data

con= sqlite3.connect("Datasets/Amazon _reviews_set/database.sqlite")

##### Filtering data

In [47]:
# Here as we only want to get the global sentiment of the recommendations (positive or negative), 
# we will purposefully ignore all Scores equal to 3. 
# If the score id above 3, then the recommendation wil be set to "positive".Otherwise, it will be set to "negative".

filtered_data = pd.read_sql_query(
"""
SELECT * 
FROM Reviews 
WHERE Score !=3 
LIMIT 10000 
""", con)
# Based on my Computationsl Power top 10k points are selected


# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

#Changing Score column to our definition
filtered_data["Score"]= list(map(partition,filtered_data["Score"]))

print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)


Number of data points in our data (10000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


### <u> Data Cleaning </u>
##### 1. Check on DeDuplication

In [60]:
# This code will Group UserId if they have same REVIEWS
# By seeing the Count Column, it gives a fair idea on how much users have duplicate Reviews

display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*) as Score
FROM Reviews
GROUP BY UserId
HAVING Score>1
""", con)

print(display.shape)
print(display.head())

#We can observe through Score that many users have duplicate reviews

(228659, 7)
               UserId   ProductId ProfileName        Time  Score  \
0  #oc-R103C0QSV1DF5E  B006Q820X0           C  1343088000      5   
1  #oc-R109MU5OBBZ59U  B008I1XPKA      AayGee  1350086400      5   
2  #oc-R10LFEMQEW6QGZ  B008I1XPKA       Julie  1345939200      5   
3  #oc-R10LT57ZGIB140  B0026LJ3EA        dipr  1310601600      3   
4  #oc-R115TNMSPFT9I7  B005ZBZLT4     Breyton  1331510400      2   

                                                Text  Score  
0  I have to say I was a little apprehensive to b...      1  
1  Received my free K cups as a sample promotion ...      1  
2  Brooklyn Bean Roastery Blend K-Cups are great ...      1  
3  The shipment of the dog food was quick.  Howev...      1  
4  Overall its just OK when considering the price...      2  


In [61]:
# Observe for Score == 5

display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*) as sum
FROM Reviews
GROUP BY UserId
HAVING sum>4
""", con)

print(display.shape)
print(display.head())

(23593, 7)
           UserId   ProductId  \
0  A1001WMV1CL0XH  B005DGI1PW   
1  A10023OS6MZUC6  B001RV8CGK   
2  A1004703RC79J9  B002TMV34E   
3  A100IC7JRCQDUD  B00016Q6BK   
4  A100UZGZNZ9ZYN  B002ANA9QA   

                                        ProfileName        Time  Score  \
0                                      Corey DePaul  1345334400      5   
1                                     Wilton Dubois  1274313600      2   
2                                        Dean Burns  1279411200      3   
3  Joel Elmer Coldwater III "Research For The Best"  1301616000      5   
4                            Nomadic PC User "Chef"  1311292800      4   

                                                Text  sum  
0  We have two small Maltese and they absolutely ...    6  
1  This taco seasoning was not good.  We used it ...    5  
2  This is good coffee, but nothing to write home...    7  
3  I use this for hygentic reasons as well as for...    7  
4  I have three cats and currently feeding ma

In [62]:
# Lets observe for UserID = A1001WMV1CL0XH as seen above

display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text
FROM Reviews
WHERE Score !=3 AND UserId ="A1001WMV1CL0XH"
ORDER BY ProductID
""", con)

print(display.shape)
print(display.head())

(6, 6)
           UserId   ProductId   ProfileName        Time  Score  \
0  A1001WMV1CL0XH  B005DGI1IY  Corey DePaul  1345334400      5   
1  A1001WMV1CL0XH  B005DGI1PW  Corey DePaul  1345334400      5   
2  A1001WMV1CL0XH  B005DGI1VG  Corey DePaul  1345334400      5   
3  A1001WMV1CL0XH  B005DGI242  Corey DePaul  1345334400      5   
4  A1001WMV1CL0XH  B005DGI2II  Corey DePaul  1345334400      5   

                                                Text  
0  We have two small Maltese and they absolutely ...  
1  We have two small Maltese and they absolutely ...  
2  We have two small Maltese and they absolutely ...  
3  We have two small Maltese and they absolutely ...  
4  We have two small Maltese and they absolutely ...  


##### 2. Remove Duplicate Data

In [ ]:
# It is observed (as shown in the table below) that the reviews data had many duplicate entries.
# Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data. 

In [ ]:
# It was inferred after analysis that reviews with same parameters other than 
# ProductId belonged to the same product just having different flavour or quantity. 
# Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.

# The method used for the same was that we first sort the data according to ProductId and 
# then just keep the first similar product review and delete the others

In [63]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [64]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(9564, 10)

In [67]:
#Checking to see how much % of data still remains
(final['Id'].size)/(filtered_data['Id'].size)*100

95.64

###### Check on Helpfulness columns (if num>den) then that data should be removed

In [ ]:
cc